In [1]:
import tensorflow as tf

batch_size = 128
num_classes = 10
epochs = 20

#tf.enable_eager_execution()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#tf.executing_eagerly()

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [5]:
import numpy as np

class MNISTSequence(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self._x_set = x_set
        self._y_set = y_set
        self._batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self._x_set) / float(self._batch_size)))
    def __getitem__(self, idx):
        batch_x = self._x_set[idx * self._batch_size:(idx + 1) * self._batch_size]
        batch_y = self._y_set[idx * self._batch_size:(idx + 1) * self._batch_size]
        
        return batch_x, batch_y

In [6]:
class MySimpleLayer(tf.keras.layers.Layer):
    def __init__(self, output_units, **kwargs):
        self.output_units = output_units
        super(MySimpleLayer, self).__init__(**kwargs)

    def build(self, input):
        self.kernel = self.add_variable(
          "kernel", [input[-1], self.output_units])
        self.bias = self.add_variable("bias", [self.output_units])
        super(MySimpleLayer, self).build(input)

    def call(self, input):
        # Override call() instead of __call__ so we can perform some bookkeeping.
        return tf.matmul(input, self.kernel) + self.bias
    
    def compute_output_shape(self, input):
        return tf.TensorShape([input[0], self.output_units])

In [7]:
# inputs = tf.keras.layers.Input(shape=x_train[0].shape)

# x = tf.keras.layers.Dense(512, activation='relu')(inputs)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# #x = tf.keras.layers.Dense(num_classes)(x)
# x = MySimpleLayer(num_classes)(x)
# x = tf.keras.layers.Activation('softmax')(x)

In [ ]:
class Model(tf.keras.models.Model):
    def __init__(self, **kwargs):
        super(Model, self).__init__(**kwargs)
    
    def call(self, x):
        x = tf.keras.layers.Dense(512, activation='relu')(inputs)
        x = tf.keras.layers.Dropout(0.2)(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        #x = tf.keras.layers.Dense(num_classes)(x)
        x = MySimpleLayer(num_classes)(x)
        x = tf.keras.layers.Activation('softmax')(x)
        return x

inputs = tf.keras.layers.Input(shape=x_train[0].shape)
outputs = Model()(inputs)

In [8]:
model = tf.keras.models.Model(inputs=inputs, outputs=x)

model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.RMSPropOptimizer(0.001),
              metrics=['accuracy'])

skip []
Tensor("activation_1_target:0", shape=(?, ?), dtype=float32)
Tensor("activation/Softmax:0", shape=(?, 10), dtype=float32)
None


In [9]:
train_gen = MNISTSequence(x_train, y_train, batch_size)
test_gen = MNISTSequence(x_test, y_test, batch_size)
model.fit_generator(train_gen, validation_data=test_gen, epochs=epochs)
# model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 2s 4ms/step - loss: 0.6458 - acc: 0.7921 - val_loss: 0.1192 - val_acc: 0.9625
Epoch 2/20
469/469 [==============================] - 1s 3ms/step - loss: 0.1191 - acc: 0.9641 - val_loss: 0.0908 - val_acc: 0.9726
Epoch 3/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0818 - acc: 0.9752 - val_loss: 0.0843 - val_acc: 0.9765
Epoch 4/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0661 - acc: 0.9800 - val_loss: 0.0761 - val_acc: 0.9796
Epoch 5/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0524 - acc: 0.9840 - val_loss: 0.0773 - val_acc: 0.9787
Epoch 6/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0470 - acc: 0.9860 - val_loss: 0.0763 - val_acc: 0.9794
Epoch 7/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0402 - acc: 0.9875 - val_loss: 0.0800 - val_acc: 0.9811
Epoch 8/20
469/469 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])